## Groupby技术

分组运算（split-apply-combine  拆分-应用-合并）

分组键可以有多种形式，且类型不必相同：
* 列表或者数组，其长度与待分组的轴一样
* 表示DataFrame某个列的值
* 字典或者Series，给出待分组轴上的值与分组名之间的对应关系
* 函数，用于处理轴索引与索引中的各个标签

In [4]:
from pandas import Series, DataFrame, Index, MultiIndex
from numpy.random import randn
import pandas as pd
import numpy as np
import datetime
import random
import re

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
df = DataFrame(
    {
        "key1": ["a", "a", "b", "b", "a"],
        "key2": ["one", "two", "one", "two", "one"],
        "data1": np.random.randn(5),
        "data2": np.random.randn(5)
    }
)

In [7]:
df

,data1,data2,key1,key2
0,0.507374,0.144276,a,one
1,-0.342977,0.505287,a,two
2,-0.501995,1.642671,b,one
3,1.756855,-1.866360,b,two
4,-0.855068,-1.289561,a,one


假设想按key1进行分组，并计算data1列的平均值。

实现方式很多，这里我们用的是：访问data1，并根据key1调用groupby

In [8]:
grouped = df["data1"].groupby(df["key1"]);grouped

grouped是一个GroupBy对象。实际上还没有进行任何计算，只是包含有一些分组键df["key1"]的中间数据而已。

In [9]:
grouped.mean()

key1
a   -0.230224
b    0.627430
Name: data1, dtype: float64

我们也可以传入多个数字，的到不同的结果。

In [10]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean();means

key1  key2
a     one    -0.173847
      two    -0.342977
b     one    -0.501995
      two     1.756855
Name: data1, dtype: float64

通过两个键对数据进行了分组，得到的Series具有一个层次化索引

In [11]:
means.unstack()

key2,one,two
key1,,
a,-0.173847,-0.342977
b,-0.501995,1.756855


上面的例子，分组键均为Series，实际上，分组键也可以是任何长度的数组

In [12]:
states = np.array(["Ohio", "California", "California", "Ohio", "Ohio"])

In [13]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [14]:
df["data1"].groupby([states, years]).mean()

California  2005   -0.342977
            2006   -0.501995
Ohio        2005    1.132114
            2006   -0.855068
Name: data1, dtype: float64

还可以将列名（字符串、数字、其他python对象）用作分组键

In [15]:
df.groupby("key1").mean()

,data1,data2
key1,,
a,-0.230224,-0.213332
b,0.627430,-0.111844


In [16]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one  -0.173847 -0.572642
     two  -0.342977  0.505287
b    one  -0.501995  1.642671
     two   1.756855 -1.866360

执行df.groupby("key1").mean()的时候，结果中并没有key2列，因为key2不是数值数据，被从结果移除了。

默认情况下，所有数值列都会被聚合。

GroupBy的size方法，可以返回一个含有分组大小的Series

In [17]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

GroupBy对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）。

In [18]:
for name, group in df.groupby("key1"):
    print name
    print group

a
      data1     data2 key1 key2
0  0.507374  0.144276    a  one
1 -0.342977  0.505287    a  two
4 -0.855068 -1.289561    a  one
b
      data1     data2 key1 key2
2 -0.501995  1.642671    b  one
3  1.756855 -1.866360    b  two


对于多重键的情况，元组的第一个元素将会由键值组成的元组

In [19]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print k1, k2
    print group

a one
      data1     data2 key1 key2
0  0.507374  0.144276    a  one
4 -0.855068 -1.289561    a  one
a two
      data1     data2 key1 key2
1 -0.342977  0.505287    a  two
b one
      data1     data2 key1 key2
2 -0.501995  1.642671    b  one
b two
      data1    data2 key1 key2
3  1.756855 -1.86636    b  two


我们可以对这些数据片段做任何操作，比如将片段做成一个字典

In [20]:
pieces = dict(list(df.groupby("key1")))

In [21]:
pieces["b"]

,data1,data2,key1,key2
2,-0.501995,1.642671,b,one
3,1.756855,-1.866360,b,two


groupby默认是在axis=0的轴上进行分组的，通过设置也可以在其他任何轴进行分组。

比如我们可以根据dtype来分组

In [22]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [23]:
grouped = df.groupby(df.dtypes, axis=1)

In [24]:
dict(list(grouped))

{dtype('float64'):       data1     data2
 0  0.507374  0.144276
 1 -0.342977  0.505287
 2 -0.501995  1.642671
 3  1.756855 -1.866360
 4 -0.855068 -1.289561, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### 选择一个或者一组列

对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或者一组（字符串组）列名对其进行索引，就能实现选取部分列进行聚合的目的。

df.groupby("key1")["data1"]

df.groupby("key1")[["data2"]]

是以下代码的语法糖

df["data1"].groupby(df["key1"])

df[["data2"]].groupby(df["key1"])

如果只需要计算data2列的平均值，并以DataFrame的形式得到结果

In [25]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    one  -0.572642
     two   0.505287
b    one   1.642671
     two  -1.866360

In [26]:
df.groupby(["key1", "key2"])["data2"].mean() # series的结果

key1  key2
a     one    -0.572642
      two     0.505287
b     one     1.642671
      two    -1.866360
Name: data2, dtype: float64

### 通过字典或者Series进行分组

In [27]:
people = DataFrame(
    randn(5,5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wes", "Jim", "Travis"]
)

In [28]:
people.ix[2:3, ["b", "c"]] = np.nan  

C:\Users\T460P\AppData\Local\conda\conda\envs\py27\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [29]:
people

,a,b,c,d,e
Joe,-1.805584,0.130722,0.935185,-0.998173,2.042606
Steve,-0.556962,0.079728,0.730945,-2.141518,0.589953
Wes,-0.791912,NaN,NaN,1.206975,0.164579
Jim,0.202472,-0.093758,1.397826,-1.888829,0.349367
Travis,-0.275042,1.952092,0.195378,0.280225,1.128944


假设已知列的分组关系，并希望根据分组进行列的总计

In [30]:
mapping = {
    "a": "red",
    "b": "red",
    "c": "blue",
    "d": "blue",
    "e": "red",
    "f": "orange"
}

In [31]:
by_column = people.groupby(mapping, axis=1)

In [32]:
by_column.sum()

,blue,red
Joe,-0.062988,0.367745
Steve,-1.410573,0.112718
Wes,1.206975,-0.627332
Jim,-0.491002,0.458081
Travis,0.475603,2.805994


Series也有同样的功能，可以被看做一个固定大小的映射。

可以根据Series进行分组

In [33]:
map_series = Series(mapping); map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [34]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 通过函数进行分组

任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值会被做分组名称。

In [35]:
people.groupby(len).sum()  # 根据人名长度

,a,b,c,d,e
3,-2.395023,0.036964,2.333011,-1.680027,2.556553
5,-0.556962,0.079728,0.730945,-2.141518,0.589953
6,-0.275042,1.952092,0.195378,0.280225,1.128944


也可以将函数跟数组、列表、字典、series混合使用

In [36]:
key_list = ["one", "one", "one", "two", "two"]

In [37]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.805584  0.130722  0.935185 -0.998173  0.164579
  two  0.202472 -0.093758  1.397826 -1.888829  0.349367
5 one -0.556962  0.079728  0.730945 -2.141518  0.589953
6 two -0.275042  1.952092  0.195378  0.280225  1.128944

### 根据索引级别分组

层次索引数据集最方便的地方在于它能够根据索引级别进行聚合。

要实现改目的，可以通过关机中level传入级别编号或者名称即可。

In [38]:
columns = pd.MultiIndex.from_arrays(
    [
        ["US", "US", "US", "JP", "JP"],
        [1, 3, 5, 1, 3]
    ],
    names=["cty", "tenor"]
)

In [39]:
hier_df = DataFrame(np.random.randn(4, 5), columns=columns);hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.968182 -2.564043 -1.590254 -0.232057 -0.322361
1     -0.026357  0.374408 -0.005774  0.061034  0.960122
2      0.615357 -0.934952  1.413092  0.933430 -0.995214
3     -0.847561  0.621691  0.799411  0.025209  0.634337

In [40]:
hier_df.groupby(level="cty", axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合

对于聚合除了使用经常用到的一些函数，mean, count, min, sum 等，

你可以使用自己发明的聚合函数，还可以调动分组对象上已经定义好的任何方法。

比如：quantile 可以计算Series或者DataFrame列的样本分位数。

In [41]:
df

,data1,data2,key1,key2
0,0.507374,0.144276,a,one
1,-0.342977,0.505287,a,two
2,-0.501995,1.642671,b,one
3,1.756855,-1.866360,b,two
4,-0.855068,-1.289561,a,one


In [42]:
grouped = df.groupby("key1")

In [43]:
grouped["data1"].quantile(0.9)

key1
a    0.337304
b    1.530970
Name: data1, dtype: float64

如果要使用自己的聚合函数，只需要将其传入aggregate或者agg方法即可

In [44]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [45]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.362442,1.794848
b,2.258850,3.509031


还要些方法如 describe也可以用到这里 

In [46]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.230224  0.688184 -0.855068 -0.599022 -0.342977  0.082198   
b      2.0  0.627430  1.597248 -0.501995  0.062717  0.627430  1.192142   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.507374   3.0 -0.213332  0.949359 -1.289561 -0.572642  0.144276   
b     1.756855   2.0 -0.111844  2.481259 -1.866360 -0.989102 -0.111844   

                          
           75%       max  
key1                      
a     0.324782  0.505287  
b     0.765413  1.642671

![groupby.png](./files/groupby.png)

为了说明一些更高级的功能，我们使用一个有关餐馆小费的数据集

In [47]:
tips = pd.read_csv("ch08/tips.csv")

In [48]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]  # 添加 “小费占总额百分比”的列

In [49]:
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


根据sex和somker对顾客进行分组

In [50]:
grouped = tips.groupby(["sex", "smoker"])

In [51]:
grouped_pct = grouped["tip_pct"]

In [52]:
grouped_pct.mean()

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

如果传入一组函数或者函数名，得到的DataFrame相应的列就会以函数名

In [53]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

如果传入的是(name, function)的元组组成的列表，则元组的第一个元素就会被当做DataFrame的列名称

In [54]:
grouped_pct.agg([("foo", "mean"), ("bar", np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

对于DataFrame，我们还可以对定义全部列、或者部分列的函数。

建设我们想统计tip_pct 和 total_bill列计算三个信息

In [55]:
functions = ["mean", "count", "max"]

In [56]:
result = grouped["tip_pct", "total_bill"].agg(functions)

In [57]:
result

tip_pct                 total_bill             
                   mean count       max       mean count    max
sex    smoker                                                  
Female No      0.156921    54  0.252672  18.105185    54  35.83
       Yes     0.182150    33  0.416667  17.977879    33  44.30
Male   No      0.160669    97  0.291990  19.791237    97  48.33
       Yes     0.152771    60  0.710345  22.284500    60  50.81

In [58]:
result["tip_pct"]

mean  count       max
sex    smoker                           
Female No      0.156921     54  0.252672
       Yes     0.182150     33  0.416667
Male   No      0.160669     97  0.291990
       Yes     0.152771     60  0.710345

跟前面一样，我们可以传入带名称的元组

In [59]:
ftuples = [("xxx", "mean"), ("ooo", np.var)]

In [60]:
grouped["tip_pct", "total_bill"].agg(ftuples)

tip_pct           total_bill           
                    xxx       ooo        xxx        ooo
sex    smoker                                          
Female No      0.156921  0.001327  18.105185  53.092422
       Yes     0.182150  0.005126  17.977879  84.451517
Male   No      0.160669  0.001751  19.791237  76.152961
       Yes     0.152771  0.008206  22.284500  98.244673

如果想对不同的列使用不同的聚合方法，可以通过字典，列名称作为key

In [61]:
grouped.agg({"tip": "max", "size": "sum"})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [62]:
grouped.agg({"tip_pct": ["mean", "std", "max"], "size": "sum"})

tip_pct                     size
                   mean       std       max  sum
sex    smoker                                   
Female No      0.156921  0.036421  0.252672  140
       Yes     0.182150  0.071595  0.416667   74
Male   No      0.160669  0.041849  0.291990  263
       Yes     0.152771  0.090588  0.710345  150

只有将多个函数应用到一列的时候，才会有层次和索引。

以无“索引”的形式返回数据，通过传入as_index=False即可

In [63]:
tips.groupby(["sex", "smoker"], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


### 分组级运算和转换

聚合只是分组的一种特例，将一维数组转换为标量的运算。

我们可以通过transform和apply来进行更多的分组操作

假设我们要为DataFrame添加一个存放各索引组平均值的列。

我们可以通过先聚合再合并的方式实现。

In [64]:
df

,data1,data2,key1,key2
0,0.507374,0.144276,a,one
1,-0.342977,0.505287,a,two
2,-0.501995,1.642671,b,one
3,1.756855,-1.866360,b,two
4,-0.855068,-1.289561,a,one


In [65]:
k1_mean = df.groupby("key1").mean().add_prefix("mean_");k1_mean

,mean_data1,mean_data2
key1,,
a,-0.230224,-0.213332
b,0.627430,-0.111844


In [66]:
pd.merge(df, k1_mean, left_on="key1", right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,0.507374,0.144276,a,one,-0.230224,-0.213332
1,-0.342977,0.505287,a,two,-0.230224,-0.213332
4,-0.855068,-1.289561,a,one,-0.230224,-0.213332
2,-0.501995,1.642671,b,one,0.627430,-0.111844
3,1.756855,-1.866360,b,two,0.627430,-0.111844


虽然上方法可以完成，但是不够灵活。

可以将该过程看做是利用np.mean函数对两个数据列进行转换。

这里我们使用transform方法来完成。

In [67]:
key = ["one", "two", "one", "two", "one"]

In [68]:
people

,a,b,c,d,e
Joe,-1.805584,0.130722,0.935185,-0.998173,2.042606
Steve,-0.556962,0.079728,0.730945,-2.141518,0.589953
Wes,-0.791912,NaN,NaN,1.206975,0.164579
Jim,0.202472,-0.093758,1.397826,-1.888829,0.349367
Travis,-0.275042,1.952092,0.195378,0.280225,1.128944


In [69]:
people.groupby(key).mean()

,a,b,c,d,e
one,-0.957512,1.041407,0.565282,0.163009,1.112043
two,-0.177245,-0.007015,1.064386,-2.015173,0.469660


In [70]:
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,-0.957512,1.041407,0.565282,0.163009,1.112043
Steve,-0.177245,-0.007015,1.064386,-2.015173,0.469660
Wes,-0.957512,1.041407,0.565282,0.163009,1.112043
Jim,-0.177245,-0.007015,1.064386,-2.015173,0.469660
Travis,-0.957512,1.041407,0.565282,0.163009,1.112043


可以看出transform会将一个函数应用到各个分组，然后将结果放置到适当位置上。

如果各分组产生的是标量值，则该值就会被广播出去。

In [71]:
def demean(arr):
    return arr - arr.mean()

In [72]:
demeaned = people.groupby(key).transform(demean)

In [73]:
demeaned

,a,b,c,d,e
Joe,-0.848071,-0.910685,0.369903,-1.161182,0.930563
Steve,-0.379717,0.086743,-0.333440,-0.126345,0.120293
Wes,0.165601,NaN,NaN,1.043966,-0.947464
Jim,0.379717,-0.086743,0.333440,0.126345,-0.120293
Travis,0.682471,0.910685,-0.369903,0.117216,0.016901


In [74]:
demeaned.groupby(key).mean()

,a,b,c,d,e
one,3.700743e-17,5.551115e-17,0.0,3.700743e-17,-7.401487e-17
two,0.000000e+00,0.000000e+00,0.0,0.000000e+00,2.775558e-17


### apply 一般性的 拆分-应用-合并

跟aggregate一样，transform也是一个有着严格限制的特殊函数：传入的函数只能产生两种结果，要么产生一个可以广播的标量，要么产生一个相同大小的结果数组。

apply会将待处理的对象拆分成多个片段，然后对各个片段调用传入函数，最后尝试将各片段合到一起。

回到之前的小费的例子，假设你要根据分组宣传最高的5个tip_pct值，首先编写一个选取指定列具有最大值得函数

In [75]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(by=column)[-n:]

In [76]:
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


对smoker进行分组，并用该函数调用apply

In [77]:
tips.groupby("smoker").apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

之前在groupby对象上调用过describe()

In [78]:
result = tips.groupby("smoker")["tip_pct"].describe()

In [79]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [80]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

禁用分组键

分组键会跟原始对象的索引共同构成结果对象的层次化索引，传入group_keys=False可以禁止该效果

In [81]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析

pandas有些能根据指定面元或者样本分位数将数据拆分成多块的工具，如cut和qcut。

将这些函数跟groupby结合起来，就可以轻松的实现对数据集的桶（bucket）和分位数（quantile）分析了。

In [82]:
frame = DataFrame({
    "data1": randn(1000),
    "data2": randn(1000)
})

In [83]:
factor = pd.cut(frame.data1, 4)

In [84]:
factor[:10]

0     (-0.238, 1.324]
1     (-0.238, 1.324]
2    (-1.799, -0.238]
3    (-1.799, -0.238]
4    (-1.799, -0.238]
5    (-1.799, -0.238]
6     (-0.238, 1.324]
7      (1.324, 2.886]
8     (-0.238, 1.324]
9     (-0.238, 1.324]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.367, -1.799] < (-1.799, -0.238] < (-0.238, 1.324] < (1.324, 2.886]]

由cut产生的factor对象可以直接用于groupby，因此我们可以对data2做一些统计操作

In [85]:
def get_stats(group):
    return {
        "min": group.min(),
        "max": group.max(),
        "count": group.count(),
        "mean": group.mean()
    }

In [86]:
grouped = frame.data2.groupby(factor)

In [87]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.367, -1.799]",48.0,2.017801,0.339361,-1.540700
"(-1.799, -0.238]",371.0,2.736471,0.002447,-2.316853
"(-0.238, 1.324]",484.0,3.913563,0.026525,-3.034705
"(1.324, 2.886]",97.0,2.219857,-0.145380,-3.230348


这些都是等长的桶。要根据样本分位数得到大小相等的桶，使用qcut即可，传入labels=False可以只获取分位数编号

In [88]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [89]:
grouped = frame.data2.groupby(grouping)

In [90]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.628159,0.052884,-2.240330
1,100.0,2.213348,-0.094231,-2.316853
2,100.0,2.372405,0.019361,-2.150277
3,100.0,2.736471,0.150223,-2.139137
4,100.0,2.207721,0.129267,-3.034705
5,100.0,3.913563,-0.014251,-2.854972
6,100.0,2.037494,0.150025,-2.672037
7,100.0,2.479559,0.074968,-2.759509
8,100.0,1.935074,-0.175582,-2.484067


### 示例： 用特定于分组的值填充缺失值

我们使用平局值来填充NA值

In [91]:
s = Series(randn(6))

In [92]:
s[::2] = np.nan

In [93]:
s

0         NaN
1    0.716429
2         NaN
3   -1.170818
4         NaN
5   -0.508516
dtype: float64

In [94]:
s.fillna(s.mean())

0   -0.320968
1    0.716429
2   -0.320968
3   -1.170818
4   -0.320968
5   -0.508516
dtype: float64

对于不同分组填充不同的值。只需要将数据分组，并使用apply和一个能够对各数据库调用fillna的函数即可。

下面是美国一些州的例子，这些州被分为东部和西部

In [95]:
states = ["Ohio", "New York", "Vermont", "Florida", "Oregon", "Nevada", "California", "Idaho"]

In [96]:
group_key = ["East"]*4 + ["West"]*4

In [97]:
data = Series(randn(8), index=states)

In [98]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan

In [99]:
data

Ohio         -0.573520
New York      0.429006
Vermont            NaN
Florida      -1.002945
Oregon       -0.128474
Nevada             NaN
California    0.377152
Idaho              NaN
dtype: float64

In [100]:
data.groupby(group_key).mean()

East   -0.382487
West    0.124339
dtype: float64

我们可以用分组平均值来填充NA值

In [101]:
fill_mean = lambda g: g.fillna(g.mean())

In [102]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.573520
New York      0.429006
Vermont      -0.382487
Florida      -1.002945
Oregon       -0.128474
Nevada        0.124339
California    0.377152
Idaho         0.124339
dtype: float64

也可以再代码中预定义各组的填充值。分组都有一个name属性，我们可以利用

In [103]:
fill_values = {"East": 0.5, "West": -1}

In [104]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [105]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.573520
New York      0.429006
Vermont       0.500000
Florida      -1.002945
Oregon       -0.128474
Nevada       -1.000000
California    0.377152
Idaho        -1.000000
dtype: float64

### 示例：随机采样和排列

假设我们要从一个大数据中取样本进行分析工作。

抽取的方法有很多，其中一些效率比其他的高。

一个办法是，选取np.random.permutation(N)的前K个元素。N为完整数据大小，K为样本大小。

下面我们构造一副扑克牌

In [106]:
suits = ["H", "S", "C", "D"]  # heart红桃 spades 黑桃          clubs梅花  diamonds方片

In [107]:
card_val = (range(1, 11) + [10]*3) * 4

In [108]:
base_names = ["A"] + range(2, 11) + ["J", "Q", "K"]

In [109]:
cards = []

In [110]:
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

In [111]:
deck = Series(card_val, index=cards)

In [112]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

In [113]:
# 从整副牌里抽取5张
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])

In [114]:
draw(deck)

6D    6
2C    2
2S    2
9S    9
8C    8
dtype: int64

假设想从每种花色抽取两种牌，由于花色是牌名最后一个字符，我们可以据此进行分组。

In [115]:
get_suit = lambda card: card[-1]

In [116]:
deck.groupby(get_suit).apply(draw, n=2)

C  2C     2
   8C     8
D  4D     4
   6D     6
H  4H     4
   2H     2
S  QS    10
   4S     4
dtype: int64

In [117]:
# 另一种方法
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

QC     10
9C      9
8D      8
3D      3
10H    10
3H      3
9S      9
AS      1
dtype: int64

### 示例： 分组加权平均数和相关系数

以下面的数据为例子，它含有分组键，值，以及权重值

In [118]:
df = DataFrame({
    "category": ["a"]*4 + ["b"]*4,
    "data": randn(8),
    "weights": np.random.rand(8)
})

In [119]:
df

,category,data,weights
0,a,-1.112513,0.653439
1,a,1.340410,0.627413
2,a,1.830871,0.114410
3,a,0.088108,0.037354
4,b,-0.973592,0.900381
5,b,-0.346916,0.619198
6,b,2.287899,0.780122
7,b,0.991915,0.811171


In [120]:
grouped = df.groupby("category")

In [121]:
get_wavg = lambda g: np.average(g["data"], weights=g["weights"])

In [122]:
grouped.apply(get_wavg)

category
a    0.228109
b    0.481550
dtype: float64

我们来点实际点的例子，下面的数据是雅虎财经的数据集，含有标准普尔500指数和几只股票的收盘价

In [123]:
close_px = pd.read_csv("ch09/stock_px.csv", parse_dates=True, index_col=0)

In [124]:
close_px[:20]

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
2003-01-09,7.34,21.93,29.44,927.57
2003-01-10,7.36,21.97,29.03,927.57
2003-01-13,7.32,22.16,28.91,926.26
2003-01-14,7.30,22.39,29.17,931.66
2003-01-15,7.22,22.11,28.77,918.22


下面计算一个由日收益率（通过百分数变化计算）与SPX之间年度相关系数组成的DataFrame

In [125]:
rets = close_px.pct_change().dropna()

In [126]:
spx_corr = lambda x: x.corrwith(x["SPX"])

In [127]:
by_year = rets.groupby(lambda x: x.year)

In [128]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


当然也可以看列与列的相关系数

In [129]:
# 苹果和微软的年度相关系数
by_year.apply(lambda g: g["AAPL"].corr(g["MSFT"]))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 示例：面向分组的线性回归

接着上个例子，我们利用regress对各个数据块执行最小二乘法回归

In [130]:
import statsmodels.api as sm

In [131]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y,X).fit()
    return result.params

按年计算AAPL对PSX收益率的线性回归

In [132]:
by_year.apply(regress, "AAPL", ["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 透视表和交叉表

透视表（pivot table）是根据一个或者多个键对数据进行聚合，并根据行和列上的分组键将数据分配到各个矩形区域中。

pandas中可以通过grupby功能以及重塑运算来制作透视表。

DataFrame有个pivot_table方法，还有顶级的pd.pivot_table函数。

除了groupby提供的便利之外，pivot_table还可以添加分项小计。

In [135]:
tips.pivot_table(index=["sex", "smoker"])

size       tip   tip_pct  total_bill
sex    smoker                                          
Female No      2.592593  2.773519  0.156921   18.105185
       Yes     2.242424  2.931515  0.182150   17.977879
Male   No      2.711340  3.113402  0.160669   19.791237
       Yes     2.500000  3.051167  0.152771   22.284500

假设我们只想聚合tip_pct和size，而且想根据day分组。将smoker放在列上，day放在行上

In [137]:
tips.pivot_table(["tip_pct", "size"], index=["sex", "day"], columns="smoker")

size             tip_pct          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   2.500000  2.000000  0.165296  0.209129
       Sat   2.307692  2.200000  0.147993  0.163817
       Sun   3.071429  2.500000  0.165710  0.237075
       Thur  2.480000  2.428571  0.155971  0.163073
Male   Fri   2.000000  2.125000  0.138005  0.144730
       Sat   2.656250  2.629630  0.162132  0.139067
       Sun   2.883721  2.600000  0.158291  0.173964
       Thur  2.500000  2.300000  0.165706  0.164417

还可以进一步处理，传入margins=True添加分项小计，这将添加all的行和列，其值对应的是单个等级中所有数据的分组统计。

下面的例子中all为平均数。

In [138]:
tips.pivot_table(["tip_pct", "size"], index=["sex", "day"], columns="smoker", margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   2.500000  2.000000  2.111111  0.165296  0.209129  0.199388
       Sat   2.307692  2.200000  2.250000  0.147993  0.163817  0.156470
       Sun   3.071429  2.500000  2.944444  0.165710  0.237075  0.181569
       Thur  2.480000  2.428571  2.468750  0.155971  0.163073  0.157525
Male   Fri   2.000000  2.125000  2.100000  0.138005  0.144730  0.143385
       Sat   2.656250  2.629630  2.644068  0.162132  0.139067  0.151577
       Sun   2.883721  2.600000  2.810345  0.158291  0.173964  0.162344
       Thur  2.500000  2.300000  2.433333  0.165706  0.164417  0.165276
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

要使用其他聚合函数，将其传给aggfunc函数即可。

如：使用count或者len可以得到分组大小的交叉表

In [139]:
tips.pivot_table(["tip_pct"], index=["sex", "smoker"], columns="day", aggfunc=len, margins=True)

tip_pct                         
day               Fri   Sat   Sun  Thur    All
sex    smoker                                 
Female No         2.0  13.0  14.0  25.0   54.0
       Yes        7.0  15.0   4.0   7.0   33.0
Male   No         2.0  32.0  43.0  20.0   97.0
       Yes        8.0  27.0  15.0  10.0   60.0
All              19.0  87.0  76.0  62.0  244.0

如果存在空的组合，你也希望设置一个fill_value

In [140]:
tips.pivot_table("size", index=["time", "sex", "smoker"], columns="day", aggfunc="sum", fill_value=0)

day                   Fri  Sat  Sun  Thur
time   sex    smoker                     
Dinner Female No        2   30   43     2
              Yes       8   33   10     0
       Male   No        4   85  124     0
              Yes      12   71   39     0
Lunch  Female No        3    0    0    60
              Yes       6    0    0    17
       Male   No        0    0    0    50
              Yes       5    0    0    23

交叉表（cross-tabulation 简称crosstab）是一种用于计算分组频率的特殊透视表。

In [142]:
data = DataFrame({
    "Sample": range(1,11),
    "Gender": ["Female", "Male", "Female", "Male","Male","Male", "Female","Female","Male", "Female"],
    "Handedness": ["Right", "Left", "Right","Right", "Left", "Right","Right", "Left", "Right","Right",]
})

In [143]:
data

,Gender,Handedness,Sample
0,Female,Right,1
1,Male,Left,2
2,Female,Right,3
3,Male,Right,4
4,Male,Left,5
5,Male,Right,6
6,Female,Right,7
7,Female,Left,8
8,Male,Right,9
9,Female,Right,10


我们想根据性别和用手习惯对数据进行汇总。虽然pivot_table可以实现，但是pd.crosstab更简单。

In [144]:
pd.crosstab(data.Gender, data.Handedness, margins=True)

Handedness,Left,Right,All
Gender,,,
Female,1,4,5
Male,2,3,5
All,3,7,10


crosstab的前两个参数可以是数组、series、数组列表。

比如在统计小费数据集

In [145]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

## 示例：2012联邦选举委员会数据库